In [24]:
def is_variable(item):
    return item.startswith('?') and all(s.isalpha() for s in item[1:])

def pattern_match(pattern, saying):
    if not pattern or not saying: 
        return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pattern_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pattern_match(pattern[1:], saying[1:])

In [25]:
pattern_match('I want ?X'.split(), "I want holiday".split())

[('?X', 'holiday')]

In [26]:
patterns = pattern_match("?X greater than ?Y".split(), "3 greater than 2".split())
print(patterns)

[('?X', '3'), ('?Y', '2')]


In [57]:
def pattern_to_dic(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}
print(pattern_to_dic(patterns))

{'?X': '3', '?Y': '2'}


In [58]:
def subsidy(rule, parsed_pattern):
    if not rule: return []
    return [parsed_pattern.get(rule[0], rule[0])] + subsidy(rule[1:], parsed_pattern)

In [59]:
got_patterns = pattern_match("I want ?X".split(), "I want iPhone".split())
#print(got_patterns)
answer = subsidy('What does it mean if you got a ?X'.split(), pattern_to_dic(got_patterns))

join_answer = ' '.join(answer)
print(join_answer)

What does it mean if you got a iPhone


In [60]:
got_pattern = pattern_match('?P needs ?X'.split(), 'John needs resting'.split())
print(got_pattern)
answer = subsidy('Why does ?P need ?X'.split(), pattern_to_dic(got_pattern))
join_answer = ' '.join(answer)
print(join_answer)

[('?P', 'John'), ('?X', 'resting')]
Why does John need resting


In [61]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}


In [62]:
import random
choice = random.choice

def get_response(saying, rules):
    if (not saying) or (not rules): return []
   
    # find the matching pattern
    for pattern, answer_pattern in defined_patterns.items():
        matched_pattern = pattern_match(pattern.split(), saying.split())
        if not matched_pattern: continue
        else:
            return ' '.join(subsidy((choice(answer_pattern)).split(), pattern_to_dic(matched_pattern)))
            
print(get_response('I need iPhone', defined_patterns))
print(get_response("My mother told me something", defined_patterns))

Image you will get iPhone soon
How do you think about your mother ?


In [183]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

def is_match(rest, saying):
    #print(rest, saying)
    if not rest or not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return ('', ''), 0

In [188]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())


(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [189]:
fail = [True, None]
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [pat, saying[0]] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        if index == 0: #no match
            return fail
        return [match]+ pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [190]:
print(pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split()))
pat_match_with_seg('?*X hello ?*Y'.split(), 'I was smart'.split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]


[True, None]

In [191]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

pat_match_with_seg('I need ?*X'.split(), 'I need an iPhone'.split())

[('?X', ['an', 'iPhone'])]

In [192]:
words = subsidy("Why do you need ?X".split(), pattern_to_dic(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))
' '.join(words)

words = subsidy("I like ?X".split(), pattern_to_dic(pat_match_with_seg('?*X is cute'.split(), 
                  "my cat is cute".split())))
' '.join(words)


'I like my cat'

In [197]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

def get_response(saying, response_rules):
    if not saying or not response_rules: return ''
    
    for pattern, response in response_rules.items():
        match = pat_match_with_seg(pattern.split(), saying.split())
        if fail == match:
            continue
        else:
            #print(match)
            return ' '.join(subsidy(choice(response).split(), pattern_to_dic(match)))
    return ''

In [198]:
get_response('I was good at table tennis', rules)

'I already knew you were good at table tennis .'

In [211]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x not ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']
}

print(get_response('Alexa, hello', rule_responses));
print(get_response('Alexa, I want a holiday', rule_responses));

How do you do
Why do you want a holiday
